## Las redes convolucionales

![title](https://miro.medium.com/max/700/1*XEPeVUd1ePhcE1-MU_eWsg.png)

Las redes neuronales convolucionales es un algoritmo de aprendizaje profundo (Deep Learning) que está diseñado para trabajar con imágenes, tomando estas como entrada (input) , asignándole importancias (pesos) a ciertos elementos en la imagen para así poder diferenciar unos de otros. Este es uno de los principales algoritmos que ha contribuido en el desarrollo y perfeccionamiento del campo de Visión por computadora.
Las redes convolucionales contienen varias capas ocultas (hidden layers), donde las primeras puedan detectar líneas, curvas y así se van especializando hasta poder reconocer formas complejas como un rostro, siluetas, etc. Las tareas comunes de este tipo de redes son:

* Detección o categorización de objetos
* Clasificación de escenas
* Clasificación de imágenes en general.

Las redes toman como entrada los pixeles de una imagen. Si tenemos una imagen con apenas 28×28 pixeles de alto y ancho, eso equivale a 784 neuronas. Y eso es si sólo tenemos 1 color (escala de grises). Si tuviéramos una imagen a color, necesitaríamos 3 canales (red, green, blue) y entonces usamos 28x28x3 = 2352 neuronas de entrada. Esa es nuestra capa de entrada.

![title](https://miro.medium.com/max/700/1*_Cb1dzBhciwRi9y1-J6qjQ.png)

Pero antes es necesario normalizar los datos, es decir que nuestros pixeles que originalmente tienen valores entre 0 y 255, convertirlos a valores entre 0 y 1, esto podemos lograrlo dividiendo cada uno de los pixeles al valor más alto que estos tienen es decir 255.

#Kernel

El kernel en las redes convolucionales se considera como el filtro que se aplica a una imagen para extraer ciertas características importantes o patrones de esta.
Por ejemplo si tenemos una imagen como la siguiente.

![title](https://miro.medium.com/max/295/1*awhD4TI4-HaJP-BrfssL1Q.png)

Aplicandole un kernel de bordes, se veria como la siguiente imagen.

![title](https://miro.medium.com/max/309/1*fiU2w-EqbKJhHoV9MDCtUg.png)

Entre las características importantes para lo que sirve el kernel son detectar bordes, enfoque, desenfoque, entre otros. Esto se logra al realizar la convolución entre la imagen y el kernel.

#La convolucion

Uno de los procesos más distintivos de estas redes son las convoluciones. El cual consiste en tomar un grupo de píxeles de la imagen de entrada e ir realizando un producto escalar con un kernel. El kernel recorrerá todas las neuronas de entrada y obtendremos una nueva matriz, la cual será una de las hidden layers. En el caso de que la imagen sea de color se tendrán 3 kernels del mismo tamaño que se sumarán para obtener una imagen de salida.

![title](https://miro.medium.com/max/700/1*67NJ1OXILVMUx-tCqaWUZg.png)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Descargando y explorando el dataset

Este dataset contiene cerca de 37,000 fotos de flores. El dataset contiene 5 sub directorios, 1 por cada clase:

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```


In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

Después de la descarga, tendrás una copia del datset a tu disposición. Hay 3,670 imágenes en total:

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

Mostremos algunas rosas

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[15]))

Y ahora algunos tulipanes

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[7]))

## Cargando el dataset con keras.preprocessing

Carguemos las imágenes usando la utilidad `image_dataset_from_directory`. Esto lo llevará de un directorio de imágenes en el disco a un **`tf.data.Dataset`** en solo un par de líneas de código.

### Creando el conjunto de datos

Definimos algunos parámetros para el cargador del dataset.

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

Es una buena práctica utilizar una división de validación al desarrollar su modelo. Usemos el 80% de las imágenes para entrenamiento y el 20% para validación.

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Los nombres de las clases los encontraremos en el atributo `class_names` de ambos conjuntos de datos. Corresponden a los nombres de los directorios en orden alfabético.

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Visualizando los datos

Mostremos las primeras 9 imágenes del dataset de entrenamiento.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Entrenaremos nuestro modelo usando estos datasets pasándoselos a la función model.fit más adelante. Antes vamos a recuperar un lote de imágenes de manera manual.

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

`image_batch` es un tensor de la forma (32, 180, 180, 3). Se trata de un lote de 32 imágenes de forma 180x180x3 (la última dimensión se refiere a los canales de color RGB). `label_batch` es un tensor de la forma (32,), estas son etiquetas correspondientes a las 32 imágenes.

Puede llamar a `.numpy()` en los tensores `image_batch` y `labels_batch` para convertirlos en un `numpy.ndarray` .

## Configurar el conjunto de datos para un mejor rendimiento.

`Dataset.cache()` mantiene las imágenes en la memoria después de que se cargan fuera del disco durante la primera época. Esto asegurará que el conjunto de datos no se convierta en un cuello de botella mientras entrena su modelo. Si su conjunto de datos es demasiado grande para caber en la memoria, también puede usar este método para crear una caché en disco de alto rendimiento.

`Dataset.prefetch()` superpone el preprocesamiento de datos y la ejecución del modelo durante el entrenamiento.

Puede obtener más información al respecto [aquí](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

val_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Estandarizar los datos

Los valores del canal RGB están en el rango [0, 255] . Esto no es ideal para una red neuronal; en general, debe intentar que los valores de entrada sean pequeños. Aquí, estandarizará los valores para que estén en el rango [0, 1] mediante el uso de una capa de cambio de escala.

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Note que los valores de los pixeles están ahora entre `[0,1]`.
print(np.min(first_image), np.max(first_image))

## Crear el modelo

El modelo consta de tres bloques de convolución con una capa de grupo máxima (maxpooling) en cada uno de ellos. Hay una capa completamente conectada (fully connected) con 128 unidades en ella que se activa mediante una función de activación relu y al final una capa densa con el número de unidades igual al número de clases. Este modelo no ha sido ajustado para una alta precisión, el objetivo de este tutorial es mostrar un enfoque estándar.

In [ ]:
num_classes = 5

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

## Compilando el modelo

Para este ejemplo usaremos como opimizador: `optimizers.Adam` y la función de pérdida `losses.SparseCategoricalCrossentropy` . Para ver la precisión del entrenamiento y la validación para cada época de entrenamiento, pasaremos el argumento de metrics .

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Resumen del modelo

Revisemos las capas del modelo:

In [ ]:
model.summary()

## Revisamos si contamos con GPU

En Colab podemos activarlo en el menú: [Entorno de ejecución] [Cambio tipo de entorno de ejecución]

In [ ]:
#Check on edit - notebook settings - gpu acceleration
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Entrenando al modelo

In [ ]:
def cpu():
  with tf.device('/cpu:0'):
    epochs=10
    history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs
    )
    return history

def gpu():
  with tf.device('/device:GPU:0'):
    epochs=10
    history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs
    )
    return history

# Elija la función dependiendo de si usará GPU o no
history = gpu() #cpu()


In [ ]:
# cpu()

## Revisemos los resultados del entrenamiento

Crearemos gráficos de pérdida y precisión en los conjuntos de entrenamiento y validación.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']
epochs = 10
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


En caso de que los valores de exactitud (accuracy) y pérdida (loss) se encuentren con un margen alto de diferencia podríamos haber caído en sobreajuste. Veamos como pudiéramos combatirlo.

## Sobreajuste (overfitting)

En los gráficos anteriores, la precisión del entrenamiento aumenta linealmente con el tiempo, mientras que la precisión de la validación se detiene alrededor del 60% en el proceso de entrenamiento. Además, la diferencia de precisión entre la precisión del entrenamiento y la validación es notable, una señal de sobreajuste .

Cuando hay una pequeña cantidad de ejemplos de entrenamiento, el modelo a veces aprende de ruidos o detalles no deseados de ejemplos de entrenamiento, hasta el punto de que impacta negativamente el rendimiento del modelo en nuevos ejemplos. Este fenómeno se conoce como sobreajuste. Significa que el modelo tendrá dificultades para generalizar en un nuevo conjunto de datos.

Hay varias formas de luchar contra el sobreajuste en el proceso de formación. En este tutorial, usará el aumento de datos y agregará Dropout a su modelo.

## Aumento de datos

El sobreajuste generalmente ocurre cuando hay una pequeña cantidad de ejemplos de entrenamiento. El aumento de datos adopta el enfoque de generar datos de entrenamiento adicionales a partir de sus ejemplos existentes al aumentarlos mediante transformaciones aleatorias que producen imágenes de aspecto creíble. Esto ayuda a exponer el modelo a más aspectos de los datos y a generalizar mejor.

Implementará el aumento de datos utilizando capas experimentales de preprocesamiento de Keras . Estos pueden incluirse dentro de su modelo como otras capas y ejecutarse en la GPU.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal",
                                                 input_shape=(img_height,
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

Revisemos cómo se ven algunos ejemplos aumentados aplicando el aumento de datos a la misma imagen varias veces:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

## Pérdida/abandono (dropout)

Otra técnica para reducir el sobreajuste es introducir Dropout en la red, una forma de regularización .

Cuando aplica Dropout a una capa, elimina aleatoriamente (estableciendo la activación en cero) una cantidad de unidades de salida de la capa durante el proceso de entrenamiento. La deserción toma un número fraccionario como valor de entrada, en la forma como 0,1, 0,2, 0,4, etc. Esto significa eliminar el 10%, 20% o 40% de las unidades de salida al azar de la capa aplicada.

`layers.Dropout`una nueva red neuronal usando `layers.Dropout`, `layers.Dropout` y luego `layers.Dropout` usando imágenes aumentadas.

In [ ]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

## Compilar y entrenar el modelo

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Revisemos ahora los nuevos resultados del entrenamiento



In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Predecir usando nuevos datos

Finalmente, usemos nuestro modelo para clasificar una imagen que no se incluyó en los conjuntos de entrenamiento o validación.

In [ ]:
sunflower_url = "https://www.latercera.com/resizer/v0X72DUmELmZrsZfVhAzoHtT9yU=/380x570/smart/arc-anglerfish-arc2-prod-copesa.s3.amazonaws.com/public/AI4BMPQE4BEFVAE7M4ARBF3SI4.jpg"
sunflower_path = tf.keras.utils.get_file('AI4BMPQE4BEFVAE7M4ARBF3SI4', origin=sunflower_url)

img = keras.preprocessing.image.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
print(score)
print(class_names)